In [0]:
%run ../utils/common

In [0]:
dbutils.widgets.text("environment", "", "")
environment = dbutils.widgets.get("environment")

In [0]:
spark.conf.set("spark.sql.autoBroadcastJoinThreshold", 314572800)

In [0]:
catalog_name = settings[environment]['catalog_name']

In [0]:
spark.sql(f"""
CREATE OR REPLACE TABLE {catalog_name}.udp_wcm_gold_vcm_dwh.d_sale_price_base_unit_daily
CLUSTER BY (product_id)  
TBLPROPERTIES (
  'DELTA.AUTOOPTIMIZE.OPTIMIZEWRITE' = 'TRUE',
  'DELTA.AUTOOPTIMIZE.AUTOCOMPACT' = 'TRUE'
) AS
WITH MAIN AS (
  SELECT /*+ REPARTITION(1000), BROADCAST(b,c) */ 
    a.calday,
    a.store_id,
    a.product_id,
    a.sale_unit,
    b.numerator,
    b.denomintr,
    b.base_uom AS base_unit,
    a.sale_price_type,
    CASE
      WHEN CAST(c.vat_code AS INT) IN (1,2,9) THEN 0
      WHEN CAST(c.vat_code AS INT) IN (3,5) THEN 0.05
      WHEN CAST(c.vat_code AS INT) IN (4) THEN 0.1
      WHEN CAST(c.vat_code AS INT) IN (6) THEN 0.08
      ELSE 0
    END vat,
    a.sale_price,
    ROUND(a.sale_price * (CASE WHEN b.numerator = 0 THEN NULL ELSE b.denomintr / b.numerator END), 0) sale_price_base_unit,
    CASE
      WHEN a.sale_unit = b.base_uom THEN 1
      WHEN b.numerator = b.denomintr THEN 2
      WHEN b.unit IS NOT NULL THEN 3
      ELSE 4
    END priority
  FROM
    {catalog_name}.udp_wcm_gold_vcm_dwh.d_sale_price_store_sku_daily a
    LEFT JOIN  {catalog_name}.udp_wcm_gold_vcm_dwh.d_product_unit b ON a.product_id = b.product_id AND a.sale_unit = b.unit
    LEFT JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_product c ON a.product_id = c.product_id
  QUALIFY ROW_NUMBER() OVER (PARTITION BY calday, store_id, a.product_id ORDER BY priority, sale_price_base_unit DESC) = 1
)

SELECT
calday, store_id, product_id, sale_unit, numerator, denomintr, base_unit, sale_price_type, vat, sale_price NUMERIC,sale_price_base_unit, CAST(ROUND(sale_price_base_unit / (1 + COALESCE(vat, 0)), 2) AS DOUBLE) AS sale_price_exc_vat, priority
FROM MAIN;
""")